<a href="https://colab.research.google.com/github/i-SanMartin/FastAI/blob/main/04_MNIST_Basics/Network_Exercise_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook fastai==2.2.5
import fastbook

In [2]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
path.ls()

(#2) [Path('/root/.fastai/data/mnist_png/training'),Path('/root/.fastai/data/mnist_png/testing')]

In [5]:
Path.BASE_PATH = path

In [6]:
(path/'training').ls()

(#10) [Path('training/2'),Path('training/5'),Path('training/8'),Path('training/7'),Path('training/6'),Path('training/0'),Path('training/9'),Path('training/3'),Path('training/4'),Path('training/1')]

In [7]:
categories = tuple(x for x in range(0,10))
categories

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [8]:
train_x = torch.tensor([])
train_y = torch.tensor([])
valid_x = torch.tensor([])
valid_y = torch.tensor([])

In [9]:
for x in categories:
  number_imgs = (path/'training'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  train_x = torch.cat([train_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  train_y = torch.cat([train_y,y])

In [10]:
for x in categories:
  number_imgs = (path/'testing'/str(x)).ls().sorted()
  zeros_tensor =  [tensor(Image.open(img)) for img in number_imgs]
  stacked_zeros = torch.stack(zeros_tensor)
  valid_x = torch.cat([valid_x,stacked_zeros])
  y = torch.tensor([x]*len(number_imgs))
  valid_y = torch.cat([valid_y,y])

In [11]:
train_x = train_x.view(-1,28*28).float()/255
valid_x = valid_x.view(-1,28*28).float()/255
train_y = train_y.unsqueeze(1).long()
valid_y = valid_y.unsqueeze(1).long()
train_x.shape, train_y.shape, valid_x.shape, valid_y.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 1]),
 torch.Size([10000, 784]),
 torch.Size([10000, 1]))

In [12]:
dset = list(zip(train_x,train_y))
dl = DataLoader(dset, batch_size=256)
valid_dset = list(zip(valid_x,valid_y))
valid_dl = DataLoader(dset, batch_size=256)
dls = DataLoaders(dl, valid_dl)

In [13]:
class optimizer():
  def __init__(self, parameters, lr):
    self.parameters = list(parameters)
    self.lr = lr
  def step(self):
    for p in self.parameters:
      p.data -= p.grad
  def grads_zero(self):
    for p in self.parameters:
      p.grad = None


In [14]:
def class_tensor_loss(size, elements, label):
  zeros = torch.zeros((size, elements))
  label = label.long()
  for _z, _l in zip(zeros, label):
    _z[_l] = 1
  return zeros

In [15]:
criterion = nn.CrossEntropyLoss()

In [ ]:
def loss_f(pred, label):
  loss_tensor = class_tensor_loss(pred.shape[0],pred.shape[1],label)
  print(loss_tensor)
  
  return (torch.abs(pred-loss_tensor)).mean()

In [16]:
def calc_grad(model,x,y):
  pred = model(x)
  loss = criterion(pred,y.squeeze(1))
  loss.backward()

In [17]:
simple_net = nn.Sequential(
    nn.Linear(28*28,500),
    nn.ReLU(),
    nn.Linear(500,100),
    nn.ReLU(),
    nn.Linear(100,30),
    nn.ReLU(),
    nn.Linear(30,len(categories)),
    nn.Sigmoid()
)

In [18]:
opt = optimizer(simple_net.parameters(), 1)

In [19]:
def batch_accuracy(x,y):
    pred = simple_net(x)
    correct = ((pred == pred.max()).nonzero(as_tuple=True)[0] == y).float().sum()

In [20]:
def validate_epoch(data):
  for xb,yb in data:
    batch_accuracy(xb,yb)

In [21]:
def train_epoch(data):
  for dx, dy in data:
    calc_grad(simple_net,dx,dy)
    opt.step()
    opt.grads_zero()

In [186]:
pr = simple_net(train_x[50009])
pr

tensor([0.4929, 0.5112, 0.4590, 0.4919, 0.5456, 0.4833, 0.5043, 0.5512, 0.4585, 0.5064], grad_fn=<SigmoidBackward>)

In [187]:
class learner():
  def __init__(self, Dls, model, optimize, loss_func, metrics):
    self.dls = Dls
    self.model = model
    self.opt = optimize
    self.loss_func = loss_func
    metrics = metrics

Streaming output truncated to the last 5000 lines.
        [5.6948e-08, 9.5127e-07, 7.2866e-08,  ..., 6.4624e-08, 4.6296e-08, 4.4226e-08]], grad_fn=<SigmoidBackward>)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor(0.1000, grad_fn=<MeanBackward0>)
tensor([[2.1901e-07, 2.7354e-06, 2.4779e-07,  ..., 2.2486e-07, 1.6886e-07, 1.6150e-07],
        [1.7959e-07, 2.2993e-06, 2.1220e-07,  ..., 1.9215e-07, 1.4248e-07, 1.3711e-07],
        [1.3771e-07, 1.8850e-06, 1.6428e-07,  ..., 1.4817e-07, 1.0828e-07, 1.0245e-07],
        ...,
        [1.9200e-09, 5.3003e-08, 2.4301e-09,  ..., 2.0266e-09, 1.5331e-09, 1.3769e-09],
        [1.4498e-06, 1.2429e-05, 1.4773e-06,  ..., 1.3968e-06, 1.0686e-06, 1.0972e-06],
        [7.2051e-10, 2.3857e-08, 9.9765e-10,  ..., 7.9513e-10, 5.8795e-10, 5.2734e-10]

In [177]:
print(train_y[50009])

tensor([8])
